In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


# Imports

In [2]:
! pip install ekphrasis
import random
import os
import re

import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torchtext import data


import matplotlib.pyplot as plt

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm

from string import punctuation
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('en')

nltk.download('punkt')
nltk.download('stopwords')

%matplotlib inline

     |████████████████████████████████| 81kB 3.0MB/s eta 0:00:011
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp36-none-any.whl size=82844 sha256=130742571903135ab89519d30f2e4a7642cc944608dcde9270343889aa602b7c
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
Successfully built ekphrasis
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Adding Custom Seed for reproducibility

In [3]:
SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
random.seed(SEED)

### Setting up pytorch's Device GPU/CPU

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

# Data

In [5]:
train_file = '/kaggle/input/nlp-getting-started/train.csv'
test_file = '/kaggle/input/nlp-getting-started/test.csv'

In [6]:
train_data = pd.read_csv(train_file)

In [7]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
print(train_data.describe())
print('NaN values in Keyword: {}'.format(train_data['keyword'].isnull().sum()))
print('NaN values in location: {}'.format(train_data['location'].isnull().sum()))

                 id      target
count   7613.000000  7613.00000
mean    5441.934848     0.42966
std     3137.116090     0.49506
min        1.000000     0.00000
25%     2734.000000     0.00000
50%     5408.000000     0.00000
75%     8146.000000     1.00000
max    10873.000000     1.00000
NaN values in Keyword: 61
NaN values in location: 2533


### First Attempt We use only text data

#### Preprocessing Data with Ekphrasis Since its Tweet Data

In [9]:
TEXT = data.Field(tokenize='spacy', include_lengths = True)
LABEL = data.LabelField(dtype=torch.long)

In [ ]:
from torchtext.data import Field, Dataset, Example
import pandas as pd

 class DataFrameDataset(Dataset):
    """Class for using pandas DataFrames as a datasource"""
    def __init__(self, examples, fields, filter_pred=None):
    """
     Create a dataset from a pandas dataframe of examples and Fields
     Arguments:
         examples pd.DataFrame: DataFrame of examples
         fields {str: Field}: The Fields to use in this tuple. The
             string is a field name, and the Field is the associated field.
         filter_pred (callable or None): use only exanples for which
             filter_pred(example) is true, or use all examples if None.
             Default is None
    """
    self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
    if filter_pred is not None:
        self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]

class SeriesExample(Example):
"""Class to convert a pandas Series to an Example"""

    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()

        for key, field in fields.items():
        if key not in data:
            raise ValueError("Specified key {} was not found in the input data".format(key))
        if field is not None:
            setattr(ex, key, field.preprocess(data[key]))
        else:
            setattr(ex, key, data[key])
        return ex
